# Description

Runs hierarchical clustering on the umap version of the data.

# Environment variables

In [ ]:
from IPython.display import display

import conf

N_JOBS = conf.GENERAL["N_JOBS"]
display(N_JOBS)

In [ ]:
%env MKL_NUM_THREADS=$N_JOBS
%env OPEN_BLAS_NUM_THREADS=$N_JOBS
%env NUMEXPR_NUM_THREADS=$N_JOBS
%env OMP_NUM_THREADS=$N_JOBS

# Modules loading

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path

import numpy as np
import pandas as pd

from utils import generate_result_set_name

# Settings

In [ ]:
np.random.seed(0)

## Input data

In [ ]:
INPUT_SUBSET = "umap"

In [ ]:
INPUT_STEM = "z_score_std-projection-smultixcan-efo_partial-mashr-zscores"

In [ ]:
# parameters of the dimentionality reduction steps
DR_OPTIONS = {
    "n_components": 50,
    "metric": "euclidean",
    "n_neighbors": 15,
    "random_state": 0,
}

In [ ]:
input_filepath = Path(
    conf.RESULTS["CLUSTERING_NULL_DIR"],
    "data_transformations",
    INPUT_SUBSET,
    generate_result_set_name(
        DR_OPTIONS, prefix=f"{INPUT_SUBSET}-{INPUT_STEM}-", suffix=".pkl"
    ),
).resolve()
display(input_filepath)

assert input_filepath.exists(), "Input file does not exist"

input_filepath_stem = input_filepath.stem
display(input_filepath_stem)

## Clustering

In [ ]:
from sklearn.cluster import AgglomerativeClustering

In [ ]:
CLUSTERING_ATTRIBUTES_TO_SAVE = ["n_clusters"]

In [ ]:
CLUSTERING_OPTIONS = {}

CLUSTERING_OPTIONS["K_MIN"] = 2
CLUSTERING_OPTIONS["K_MAX"] = 75  # sqrt(3749) + some more to get closer to 295
CLUSTERING_OPTIONS["LINKAGE"] = {"ward", "complete", "average", "single"}
CLUSTERING_OPTIONS["AFFINITY"] = "euclidean"

display(CLUSTERING_OPTIONS)

In [ ]:
CLUSTERERS = {}

idx = 0

for k in range(CLUSTERING_OPTIONS["K_MIN"], CLUSTERING_OPTIONS["K_MAX"] + 1):
    for linkage in CLUSTERING_OPTIONS["LINKAGE"]:
        if linkage == "ward":
            affinity = "euclidean"
        else:
            affinity = "precomputed"

        clus = AgglomerativeClustering(
            n_clusters=k,
            affinity=affinity,
            linkage=linkage,
        )

        method_name = type(clus).__name__
        CLUSTERERS[f"{method_name} #{idx}"] = clus

        idx = idx + 1

In [ ]:
display(len(CLUSTERERS))

In [ ]:
_iter = iter(CLUSTERERS.items())
display(next(_iter))
display(next(_iter))

In [ ]:
clustering_method_name = method_name
display(clustering_method_name)

## Output directory

In [ ]:
# output dir for this notebook
RESULTS_DIR = Path(
    conf.RESULTS["CLUSTERING_NULL_DIR"],
    "runs",
    f"{INPUT_SUBSET}-{INPUT_STEM}",
).resolve()
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

display(RESULTS_DIR)

# Load input file

In [ ]:
data = pd.read_pickle(input_filepath)

In [ ]:
data.shape

In [ ]:
data.head()

In [ ]:
assert not data.isna().any().any()

# Clustering

## Generate ensemble

In [ ]:
from sklearn.metrics import pairwise_distances
from clustering.ensembles.utils import generate_ensemble

In [ ]:
data_dist = pairwise_distances(data, metric=CLUSTERING_OPTIONS["AFFINITY"])

In [ ]:
data_dist.shape

In [ ]:
pd.Series(data_dist.flatten()).describe().apply(str)

In [ ]:
ensemble = generate_ensemble(
    data_dist,
    CLUSTERERS,
    attributes=CLUSTERING_ATTRIBUTES_TO_SAVE,
    affinity_matrix=data_dist,
)

In [ ]:
# the number should be close to 295 (the number of partitions generated by k-means/spectral clustering)
ensemble.shape

In [ ]:
ensemble.head()

In [ ]:
ensemble["n_clusters"].value_counts().head()

In [ ]:
ensemble_stats = ensemble["n_clusters"].describe()
display(ensemble_stats)

### Testing

In [ ]:
assert ensemble_stats["min"] > 1

In [ ]:
assert not ensemble["n_clusters"].isna().any()

In [ ]:
assert ensemble.shape[0] == len(CLUSTERERS)

In [ ]:
# all partitions have the right size
assert np.all(
    [part["partition"].shape[0] == data.shape[0] for idx, part in ensemble.iterrows()]
)

In [ ]:
# no partition has negative clusters (noisy points)
assert not np.any([(part["partition"] < 0).any() for idx, part in ensemble.iterrows()])

## Save

In [ ]:
del CLUSTERING_OPTIONS["LINKAGE"]

output_filename = Path(
    RESULTS_DIR,
    generate_result_set_name(
        CLUSTERING_OPTIONS,
        prefix=f"{clustering_method_name}-",
        suffix=".pkl",
    ),
).resolve()
display(output_filename)

In [ ]:
ensemble.to_pickle(output_filename)